### Import Libraries

In [5]:
import streamlit as st
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings
from pymongo import MongoClient
import config
from langchain_community.llms import Ollama

In [8]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus
import pandas as pd
import json

username = quote_plus('<ENTER_YOUR_USERNAME>')
password = quote_plus('<ENTER_YOUR_PASSWORD>')
cluster = '<ENTER_YOUR_CLUSTER>'

mongo_uri = "mongodb+srv://" + username + ":" + password + "@" + cluster + ".ypsyghd.mongodb.net/?retryWrites=true&w=majority&appName=" + cluster
db_name = "<ENTER_YOUR_DB>"
coll_name = "<ENTER_YOUR_COLLECTION>"

### Insert JSON file into MongoDB Atlas with vector embeddings
##### 1. Using a smaller number of JSON file for testing

In [35]:
from sentence_transformers import SentenceTransformer
import pymongo
import config
import os
import json

mongo_uri = mongo_uri
db = db_name
collection = coll_name

# initialize db connection
connection = pymongo.MongoClient(mongo_uri)
collection = connection[db][collection]

# Define the path to the directory containing JSON files
directory_path = 'earning_call_transcripts_json_test'

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Process each JSON file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            for record in data:
                content = record.get("content", "")
                if content:
                    # Compute the embedding
                    embedding = model.encode(content).tolist()  # Convert to list for MongoDB compatibility
                    # Prepare document to insert
                    document = {
                        "filename": filename,
                        "content": content,
                        "embedding": embedding
                    }
                    # Insert document into MongoDB
                    collection.insert_one(document)
                    print(f"Inserted document for {filename} into MongoDB.")

# Note: Ensure that the MongoDB URI, database, and collection names are correctly configured.

Inserted document for AAPL.json into MongoDB.
Inserted document for MDB.json into MongoDB.
Inserted document for MDB.json into MongoDB.
Inserted document for MDB.json into MongoDB.
Inserted document for MDB.json into MongoDB.
Inserted document for MSFT.json into MongoDB.
Inserted document for MSFT.json into MongoDB.
Inserted document for NVDA.json into MongoDB.
Inserted document for NVDA.json into MongoDB.
Inserted document for NVDA.json into MongoDB.
Inserted document for NVDA.json into MongoDB.
Inserted document for QCOM.json into MongoDB.


### Alternate code to ensure embedding is performed

In [38]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Process documents to compute embeddings
for document in collection.find({"content_embedding": {"$exists": False}}).limit(10):
    content = document.get("content", "")
    if content:  # Check if content is not empty
        print("Computing vector for document ID:", document["_id"])
        embedding = model.encode(content).tolist()  # Convert to list for MongoDB compatibility
        # Update the document with the computed embedding
        collection.update_one(
            {"_id": document["_id"]},
            {"$set": {"content_embedding": embedding}},
            upsert=True
        )
        print("Vector computed and stored for document ID:", document["_id"])
    else:
        print("No content available for document ID:", document["_id"])
